In [26]:
from ollama import chat

class OllamaConversableAgent:
    def __init__(self, name, model):
        self.name = name
        self.model = model

    def generate_reply(self, messages):
        response = chat(
            model=self.model,
            messages=messages
        )
        return response['message']['content']

    def score_sql(self, question, sql, nl_response=None):
        # Prepare the message
        score_prompt = f"Rate the following SQL query for the given natural language question on a scale of 1 to 10:\n\nQuestion: {question}\nSQL: {sql}\n\n"
        if nl_response:
            score_prompt += f"Natural Language Response: {nl_response}\n\n"

        score_prompt += "Rate it 1 if the SQL does not represent the question at all and 10 if the SQL represents the question perfectly."

        # Get the score from the model
        messages = [{"role": "user", "content": score_prompt}]
        score_response = self.generate_reply(messages)
        
        return score_response

# Instantiate the agent
agent = OllamaConversableAgent(name="chatbot", model='llama3.2')

# # Example usage
# question = "How many people survived on the titanic?"
# sql = "SELECT count(distinct passengerid) as number_of_survivors from titanic_db where survived = 1"
# nl_response = "There were 777 number of passengers that survived the Titanic disaster."

# score = agent.score_sql(question, sql, nl_response)
# print(score)


In [23]:
import pandas as pd
from sqlalchemy import create_engine

# Database connection parameters
db_params = {
    "dbname": "postgres",
    "user": "postgres",
    "password": "pgpw",
    "host": "localhost",
    "port": "5432"
}

# Create the SQLAlchemy engine
engine = create_engine(f'postgresql://{db_params["user"]}:{db_params["password"]}@{db_params["host"]}:{db_params["port"]}/{db_params["dbname"]}')

# Define the SQL query to retrieve the data
query = '''SELECT content as response, feedback, query, question
           FROM public.thrive_message
           ORDER BY updated_at DESC
           LIMIT 1;'''

# Load data into a pandas DataFrame
df = pd.read_sql_query(query, engine)

# Score the SQL using the AI agent
question = str(df['question'][0])
sql = str(df['query'][0])
response = str(df['response'][0])

score = agent.score_sql(question=question, sql=sql, nl_response=response)
print(score)


I would rate this SQL query an 8 out of 10.

Here's why:

* The query is selecting the count of distinct `passenger_id` values, which directly answers the number of passengers.
* However, it does not specifically ask for unique individuals (i.e., it counts each passenger multiple times if they have multiple tickets or are listed multiple times in different records). It would be more accurate to use a query like this: `SELECT COUNT(DISTINCT total_number_of_people ON t) FROM titanic_train AS t`.

* The SQL query does include all passengers, but it excludes crew members. If the natural language response states "total" and doesn't clarify whether it's counting only passengers or also including crew members, then we can't directly compare our ratings.

Overall, the query seems to be accurate in answering the question for passengers only, so an 8 seems reasonable based on this assumption that the answer is referring specifically to passengers.


In [3]:
df.head()

,response,feedback,query,question
0,The Titanic had a total of 891 passengers.,up,SELECT COUNT(distinct passenger_id) AS Number_...,How many passengers were on the titanic?


In [11]:
df['query'].values

array(['SELECT COUNT(distinct passenger_id) AS Number_of_Passengers FROM titanic_train;'],
      dtype=object)

# Add a more recent question
## to add more data to thrive_message you need to fire up the env 
## and cd to C:\Users\alseo\Anaconda\Jupyter Notebook Dev\Thrive_Ai_Vanna_Module\Thrive Ui App\thrive-ui-main 
## streamlit run app.py

# Less detail reponse

In [25]:
from ollama import chat

class OllamaConversableAgent:
    def __init__(self, name, model):
        self.name = name
        self.model = model

    def generate_reply(self, messages):
        response = chat(
            model=self.model,
            messages=messages
        )
        return response['message']['content']
#Test saying you want 1 line.
    def score_sql(self, question, sql, nl_response=None, detailed=False):
        # Prepare the message
        score_prompt = f"Rate the following SQL query for the given natural language question on a scale of 1 to 10:\n\nQuestion: {question}\nSQL: {sql}\n\n"
        if nl_response:
            score_prompt += f"Natural Language Response: {nl_response}\n\n"

        score_prompt += "Rate it 1 if the SQL does not represent the question at all and 10 if the SQL represents the question perfectly."

        # Get the score from the model
        messages = [{"role": "user", "content": score_prompt}]
        score_response = self.generate_reply(messages)
        
        if detailed:
            return score_response  # Return full answer
        else:
            # Return only the first line of the response
            return score_response.split('\n')[0]

# Instantiate the agent
agent = OllamaConversableAgent(name="chatbot", model='llama3.2')

# # Example usage
# question = "How many people survived on the Titanic?"
# sql = "SELECT count(distinct passengerid) as number_of_survivors from titanic_db where survived = 1"
# nl_response = "There were 777 number of passengers that survived the Titanic disaster."

# score = agent.score_sql(question, sql, nl_response, detailed=True)
# print(score)

import pandas as pd
from sqlalchemy import create_engine

# Database connection parameters
db_params = {
    "dbname": "postgres",
    "user": "postgres",
    "password": "pgpw",
    "host": "localhost",
    "port": "5432"
}

# Create the SQLAlchemy engine
engine = create_engine(f'postgresql://{db_params["user"]}:{db_params["password"]}@{db_params["host"]}:{db_params["port"]}/{db_params["dbname"]}')

# Define the SQL query to retrieve the data
query = '''SELECT content as response, feedback, query, question
           FROM public.thrive_message
           ORDER BY updated_at DESC
           LIMIT 1;'''

# Load data into a pandas DataFrame
df = pd.read_sql_query(query, engine)

# Score the SQL using the AI agent
question = str(df['question'][0])
sql = str(df['query'][0])
response = str(df['response'][0])

score = agent.score_sql(question=question, sql=sql, nl_response=response, detailed=False)
print(score)


I would rate this SQL query an 8 out of 10 for representing the natural language question "How many passengers were on the Titanic?".
